## Text Generation

Using deep learning model to generate Harry Potter-liked stories . Given a sequence of words as seed, this model predicts the following words.

#### Import required python packages

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers
import string
import time
from random import randint
import numpy as np
from pickle import dump, load
from gensim.models import Word2Vec
import re
import os
import tensorflow as tf

print(tf.__version__)

2.3.1


#### Prepare the training datasets

In [2]:
# Prepare dataset
directory = 'HarryPotter-en/'
fnames = os.listdir(directory)
fnames.sort()

txt_data_files = []
for f in fnames:
    txt_data_files.append(os.path.join(directory, f))
txt_data_files

['HarryPotter-en/Harry-Potter-and-the-Chamber-of-Secrets.txt',
 'HarryPotter-en/Harry-Potter-and-the-Deathly-Hallows.txt',
 'HarryPotter-en/Harry-Potter-and-the-Goblet-of-Fire.txt',
 'HarryPotter-en/Harry-Potter-and-the-Half-Blood-Prince.txt',
 'HarryPotter-en/Harry-Potter-and-the-Order-of-the-Phoenix.txt',
 'HarryPotter-en/Harry-Potter-and-the-Philosophers-Stone.txt',
 'HarryPotter-en/Harry-Potter-and-the-Prisoner-of-Azkaban.txt']

In [3]:
# load text data
def load_txt_data(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [4]:
# load dataset
txt_data_set = ''
for f in txt_data_files:
    txt_data_set += load_txt_data(f)
    txt_data_set = load_txt_data(f)
print(txt_data_set[:200])

Harry Potter was a highly unusual boy in many ways. For one thing, he hated the summer holidays more than any other time of year. For another, he really wanted to do his homework but was forced to do 


#### Pre-process the text data

1. replace '--' with a space ' '.
2. tokenize the text by whits space
3. remove puntuation from each token
4. remove all tokens that are not alphabetic
5. convert to lower case

In [5]:
def preprocess_txt_dataset(txt_data):
    txt_data = txt_data.replace('--', ' ')  # replace '--' with a space ' '
    txt_data = txt_data.lower().strip()
    # creating a space between a word and the punctuation following it    
    #txt_data = re.sub(r"([?.!,¿])", r" \1 ", txt_data)
    #txt_data = re.sub(r'[" "]+', " ", txt_data)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    #txt_data = re.sub(r"[^a-zA-Z?.!,¿]+", " ", txt_data)
    txt_data = txt_data.rstrip().strip()
    
    tokens = txt_data.split()  # split into tokens by white space
    table = str.maketrans('', '', string.punctuation)  # remove punctuation from each token
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word.isalpha()]  # remove remaining tokens that are not alphabetic
    #tokens = [word.lower() for word in tokens]  # make lower case
    return tokens

In [6]:
# Pre-process and tokenize document
tokens = preprocess_txt_dataset(txt_data_set)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['not', 'for', 'the', 'first', 'time', 'an', 'argument', 'had', 'broken', 'out', 'over', 'breakfast', 'at', 'number', 'four', 'privet', 'drive', 'mr', 'vernon', 'dursley', 'had', 'been', 'woken', 'in', 'the', 'early', 'hours', 'of', 'the', 'morning', 'by', 'a', 'loud', 'hooting', 'noise', 'from', 'his', 'nephew', 'room', 'time', 'this', 'he', 'roared', 'across', 'the', 'table', 'you', 'control', 'that', 'owl', 'have', 'to', 'harry', 'tried', 'yet', 'again', 'to', 'explain', 'he', 'said', 'used', 'to', 'flying', 'around', 'outside', 'if', 'i', 'could', 'just', 'let', 'her', 'out', 'at', 'i', 'look', 'snarled', 'uncle', 'vernon', 'a', 'bit', 'of', 'fried', 'egg', 'dangling', 'from', 'his', 'bushy', 'mustache', 'know', 'happen', 'if', 'that', 'let', 'he', 'exchanged', 'dark', 'looks', 'with', 'his', 'wife', 'petunia', 'harry', 'tried', 'to', 'argue', 'back', 'but', 'his', 'words', 'were', 'drowned', 'by', 'a', 'long', 'loud', 'belch', 'from', 'the', 'son', 'dudley', 'want', 'more', 'more'

Organize the text data into sequences of tokens. Each sequence has a length of 50+1 tokens.

In [7]:
# organize into sequences of tokens
length = 50 + 1
#length = 7 + 1
txt_sequences = list()

for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    txt_sequences.append(line)
print('Total Sequences: %d' % len(txt_sequences))

Total Sequences: 448776


In [8]:
def save_txt_data(txt_lines, filename):
    text = '\n'.join(txt_lines)
    file = open(filename, 'w')
    file.write(text)
    file.close()

In [9]:
# save sequences to file
out_filename = 'HarryPotter_series.txt'
save_txt_data(txt_sequences, out_filename)

#### Tokenize

Tokenize the text sequences by Tokenizer of keras. The output is sequences of word index.

In [10]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(txt_sequences)
sequences = tokenizer.texts_to_sequences(txt_sequences)

There are total 7731 vocablaries.

In [11]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

16108


Prepare word sequences for gensim word2vec model training. The word_sequences is a 2D array of tokens.

In [12]:
word_sequences = []

for i in range(len(txt_sequences)):
    word_sequences.append(txt_sequences[i].split())

In [13]:
print(word_sequences[0:2])

[['not', 'for', 'the', 'first', 'time', 'an', 'argument', 'had', 'broken', 'out', 'over', 'breakfast', 'at', 'number', 'four', 'privet', 'drive', 'mr', 'vernon', 'dursley', 'had', 'been', 'woken', 'in', 'the', 'early', 'hours', 'of', 'the', 'morning', 'by', 'a', 'loud', 'hooting', 'noise', 'from', 'his', 'nephew', 'room', 'time', 'this', 'he', 'roared', 'across', 'the', 'table', 'you', 'control', 'that', 'owl', 'have'], ['for', 'the', 'first', 'time', 'an', 'argument', 'had', 'broken', 'out', 'over', 'breakfast', 'at', 'number', 'four', 'privet', 'drive', 'mr', 'vernon', 'dursley', 'had', 'been', 'woken', 'in', 'the', 'early', 'hours', 'of', 'the', 'morning', 'by', 'a', 'loud', 'hooting', 'noise', 'from', 'his', 'nephew', 'room', 'time', 'this', 'he', 'roared', 'across', 'the', 'table', 'you', 'control', 'that', 'owl', 'have', 'to']]


In [14]:
embedding_dim = 256  # embedding dimension of word2vec model

#### gensim model training

In [15]:
gensim_model = Word2Vec(word_sequences, size = embedding_dim, iter=5, sg=1, window=5, min_count=5, max_vocab_size=None, hs=0, negative=5, workers=3)

In [16]:
gensim_model.save('word2vec.model')

In [17]:
gensim_model = Word2Vec.load('word2vec.model')

Read the word vectors trained by gensim and prepare an embedding matrix for the initialization of Embedding layer.

In [18]:
# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for w in gensim_model.wv.vocab:
    word_vectors = np.array([gensim_model[w]])
    embedding_matrix[tokenizer.word_index[w]] = word_vectors

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Test the performance of the model trained by gensim.

In [19]:
gensim_model.most_similar('one')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('on', 0.3945864737033844),
 ('that', 0.38677656650543213),
 ('the', 0.3813679814338684),
 ('all', 0.37862300872802734),
 ('it', 0.37540748715400696),
 ('had', 0.3650192618370056),
 ('and', 0.35878318548202515),
 ('at', 0.35357147455215454),
 ('for', 0.34648340940475464),
 ('who', 0.3339601755142212)]

In [20]:
gensim_model.similarity('harry', 'wizard')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.12400773

Prepare the training dataset and labels

In [21]:
# separate into training data and label
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)  # One-hot encoding
seq_length = X.shape[1]

In [22]:
seq_length

50

Set the hyper-parameters

In [23]:
memory_cell = 256
dropout_rate = 0.2

 #### Define a deep learning model

Define a deep learning sequential model with one embedding layer initialized by the word vectors pre-trained by gensim, two LSTM layers with 256 memory cells, one dropout layer with dropout rate 0.2, one dense layer with 256 neurons and relu activation, another dropout layer with dropout rate 0.2, and one output dense layer with 7731 neurons and softmax activation.

In [24]:
# define model
harry_potter_model = Sequential()
harry_potter_model.add(Embedding(vocab_size, embedding_dim, input_length=seq_length, 
    embeddings_initializer=initializers.Constant(embedding_matrix), trainable=False,))
harry_potter_model.add(LSTM(memory_cell, return_sequences=True))
harry_potter_model.add(LSTM(memory_cell))
harry_potter_model.add(Dropout(dropout_rate))
harry_potter_model.add(Dense(memory_cell, activation='relu'))
harry_potter_model.add(Dropout(dropout_rate))
harry_potter_model.add(Dense(vocab_size, activation='softmax'))
harry_potter_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 256)           4123648   
_________________________________________________________________
lstm (LSTM)                  (None, 50, 256)           525312    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16108)             4

In [25]:
# compile model
harry_potter_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Set the hyper-parameters.

In [26]:
batch_size = 128
epochs = 400

#### Train the Harry-Potter style text generator model

In [27]:
tick1 = time.time()
localtime = time.asctime(time.localtime(time.time()))
print("開始時間是：", localtime)

# fit model
harry_potter_model.fit(X, y, validation_split=0.2, batch_size=batch_size, epochs=epochs)

localtime = time.asctime(time.localtime(time.time()))
print("結束時間是：", localtime)

tick2 = time.time()
print("Elapsed time: ", tick2 - tick1)
print(time.strftime("%H:%M:%S", time.localtime(tick2-tick1-3600*8)))

開始時間是： Fri Oct 16 14:07:17 2020
結束時間是： Fri Oct 16 14:07:17 2020
Elapsed time:  0.0021424293518066406
00:00:00


In [28]:
# save the model to file
harry_potter_model.save('harry_potter_model.h5')
# save the tokenizer
dump(tokenizer, open('harry_potter_tokenizer.pkl', 'wb'))

In [29]:
seq_length = len(txt_sequences[0].split()) - 1
print(seq_length)

50


In [30]:
# load the model
harry_potter_model = load_model('harry_potter_model.h5')
# load the tokenizer
tokenizer = load(open('harry_potter_tokenizer.pkl', 'rb'))

#### Stochastic sampling

Stochastic sampling introduces randomness in the sampling process, by sampling from
the probability distribution for the next word. In order to control the amount of stochasticity in the sampling process, a parameter called the softmax temperature that characterizes the entropy of the probability distribution is introduced. Given a temperature value, a new probability distribution is computed from the predicted probability distribution of the model. Higher temperature results in higher entropy of the new probability distribution and generates more creative text data. On the contrary, lower temperature results in less randomness of the distribution and generates much more predictable data.

In [31]:
def txt_gen_sample(preds, temperature=0.5):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [32]:
# generate a sequence from a language model
def generate_txt_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    
    # generate n_words of words
    for _ in range(n_words):
        # tokenize the seed text sequence
        encoded = tokenizer.texts_to_sequences([seed_text])[0]

        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, padding='pre', truncating='pre')
        # predict probabilities of the next word
        preds = model.predict(encoded, verbose=0)
        # Stochastic sampling with temperature 0.5
        idx = txt_gen_sample(preds[0], 0.5)
        
        # map the predicted index to word
        out_word = tokenizer.index_word[idx]
        
        # append to the seed text
        seed_text += ' ' + out_word
        # append to the generated text sequence
        result.append(out_word)

    return ' '.join(result)

Prepare the seed text.

In [33]:
directory = 'generated_story/'

if os.path.isfile('generated_story/gen_story1.txt'):
    os.remove('generated_story/gen_story1.txt')
if os.path.isfile('generated_story/gen_story2.txt'):
    os.remove('generated_story/gen_story2.txt')
if os.path.isfile('generated_story/gen_story3.txt'):
    os.remove('generated_story/gen_story3.txt')
    
fnames = os.listdir(directory)
fnames.sort()
seed_txt_files = []
for f in fnames:
    seed_txt_files.append(os.path.join(directory, f))
seed_txt_files

['generated_story/story1.txt',
 'generated_story/story2.txt',
 'generated_story/story3.txt']

Generate 250 words for each seed text.

In [34]:
num_words = 250

for i, s in enumerate(seed_txt_files):
    seed_text = load_txt_data(s)
    #generate new text
    generated_seq = generate_txt_seq(harry_potter_model, tokenizer, seq_length, seed_text, num_words)
    
    # Save to file
    file = open(directory+'gen_'+fnames[i], 'w')
    file.write(seed_text+generated_seq)
    file.close()
    print(f'Story: {i+1}')
    print(f'Seed Text: {seed_text}')
    print(f'Generated Story: {seed_text+generated_seq}\n')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


Story: 1
Seed Text: mr dursley might have been drifting into an uneasy sleep

Generated Story: mr dursley might have been drifting into an uneasy sleep
he wished theyd acted remotely of guilt sticking up to the top of the common room the chamber of secrets has been given mum and dad said him as though this question was cracked and woken that was moving to the rest of the school apparently expected to encounter the diet of the boy who was crying asleep before the grave to the lightning scar on forehead but sense he sat at ron and harry together ron they hurried away from the wizards who looked at each other in celebration watching the gryffindors pavement more odd faces you cars in october five minutes or so the goblin minister exploded in her hand and continued to glare in turning think stupid lets try and give me a quick word i dozed yer staying to get past a dragon to stop the slytherins to the floo network wizarding creatures the class may be revealed to warn that a witch has been p